In [1]:
##crear la cadena de conexion para conectarnos a mysql(le indicamos a que base de datos nos conectamos)
import src 
conn_ = src.get_conn_string('./password.txt','videoclub')
###incluir el txt del password en gitignore


In [2]:
from sqlalchemy import create_engine

In [3]:
#Entrar en la base de datos
cursor=create_engine(conn_)

In [4]:
import pandas as pd

#importamos los datos

actor=pd.read_csv('../data/actor_clean.csv')
category=pd.read_csv('../data/category_clean.csv')
film=pd.read_csv('../data/film_clean.csv')
inventory=pd.read_csv('../data/inventory_clean.csv')
language=pd.read_csv('../data/language_clean.csv')
old=pd.read_csv('../data/old_clean.csv')
rental=pd.read_csv('../data/rental_clean.csv')

In [5]:
#Concatenar nombre + apellido en las tablas actor y old para poder relacionarlas

actor['full_name'] = actor['first_name'] + ' ' + actor['last_name']

old['full_name'] = old['first_name'] + ' ' + old['last_name']

#Merge de la tabla de old y actores

old_actor=pd.merge(old,actor,on="full_name", how="left")

old_actor.head()

,first_name_x,last_name_x,title,release_year,category_id,full_name,actor_id,first_name_y,last_name_y,last_update
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6,PENELOPE GUINESS,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2,PENELOPE GUINESS,1,PENELOPE,GUINESS,2006-02-15 04:34:33
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13,PENELOPE GUINESS,1,PENELOPE,GUINESS,2006-02-15 04:34:33
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10,PENELOPE GUINESS,1,PENELOPE,GUINESS,2006-02-15 04:34:33
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14,PENELOPE GUINESS,1,PENELOPE,GUINESS,2006-02-15 04:34:33


In [6]:
#borrar columnas duplicadas
old_actor.drop(['first_name_x','last_name_x','first_name_y','last_name_y','last_update'], axis=1, inplace=True)
old_actor

,title,release_year,category_id,full_name,actor_id
0,ACADEMY DINOSAUR,2006,6,PENELOPE GUINESS,1
1,ANACONDA CONFESSIONS,2006,2,PENELOPE GUINESS,1
2,ANGELS LIFE,2006,13,PENELOPE GUINESS,1
3,BULWORTH COMMANDMENTS,2006,10,PENELOPE GUINESS,1
4,CHEAPER CLYDE,2006,14,PENELOPE GUINESS,1
...,...,...,...,...,...
995,COMANCHEROS ENEMY,2006,3,GOLDIE BRODY,39
996,DAISY MENAGERIE,2006,14,GOLDIE BRODY,39
997,DESERT POSEIDON,2006,11,GOLDIE BRODY,39
998,EVERYONE CRAFT,2006,9,GOLDIE BRODY,39


In [7]:
#Crear tabla actor-pelicula
#Merge de la tabla de old_actor y film

old_actor_film=pd.merge(old_actor,film,on="title", how="left")

#borrar columnas para dejar una tabla de id
old_actor_film.drop(['title','release_year_x','full_name','description','release_year_y','rental_duration',
               'rental_rate','length','replacement_cost','rating','special_features',
                'last_update','language_id'], axis=1, inplace=True)

#pd.set_option('display.max_rows', None)

old_actor_film = old_actor_film[['film_id','actor_id','category_id']]
old_actor_film

,film_id,actor_id,category_id
0,1,1,6
1,23,1,2
2,25,1,13
3,106,1,10
4,140,1,14
...,...,...,...
995,168,39,3
996,203,39,14
997,222,39,11
998,290,39,9


In [8]:
##cargar las tablas en la base de datos
actor.to_sql(name='actor', con=cursor, if_exists='replace', index=False)
category.to_sql(name='category', con=cursor, if_exists='replace', index=False)
film.to_sql(name='film', con=cursor, if_exists='replace', index=False)
inventory.to_sql(name='inventory', con=cursor, if_exists='replace', index=False)
language.to_sql(name='language', con=cursor, if_exists='replace', index=False)
old_actor_film.to_sql(name='old_actor_film', con=cursor, if_exists='replace', index=False)
rental.to_sql(name='rental', con=cursor, if_exists='replace', index=False)

1000